# Решение задачи "Анализ ЭКГ-сигналов для диагностики сердечных патологий" для конкурса AI challenge

Оглавление
- Исследование области задачи
- Обработка и анализ данных
- Тестирование различных моделей

## Исследование области задачи
- SCP-ECG - http://masters.donntu.ru/2008/kita/golovach/library/4_ref/pub.html
- стадии инфаркта

## Обработка и анализ данных

Используемые модули

In [ ]:
# Для данных
import pandas as pd
import numpy as np

# Для плюшек
import sklearn as sk

# Для красоты
import seaborn as sns
from pprint import pprint

Полезные функции

In [ ]:
def get_hr(folder: str, hr_num: str):
    return np.load(f'data/{folder}/{hr_num}.npy')

Пример использования

In [ ]:
a = get_hr(folder='train', hr_num='15857_hr')
sns.lineplot(data=a.flatten()[:1500])

Загрузка данных

In [ ]:
meta = pd.read_csv('data/train/train_meta.csv')
diagnosis = pd.read_csv('data/train/train_gts.csv')

In [ ]:
meta.head(3)

In [ ]:
diagnosis.head(3)

Просмотрим инофрмацию о нашем датасете </br>
https://physionet.org/content/ptb-xl/1.0.3/ - еще здесь нада

In [ ]:
meta.info()

Удалим те данные, которые не несут важной для нас информации

In [ ]:
useless_columns = ['ecg_id', 'patient_id', 'nurse', 'site', 'device', 'recording_date', 'filename_lr', 'filename_hr']
meta.drop(columns=useless_columns, inplace=True)

Создадим список тех столбцов, которые возможно не несут важной информации (мы проверим это при обучении)

In [ ]:
strange_columns = ['age', 'sex', 'group']

БОНУС интересный пациент

In [ ]:
print(meta.iloc[1514]) # интересный пациент
print()
print(diagnosis.loc[diagnosis['record_name'] == '15857_hr']) # без миокарда
# Скорее всего его придется убрать, т.к. он почти пустой, хотя желательно это оптимизировать и убрать большинство таких
hrs = get_hr('train', '15857_hr')

Рассмотрим те столбцы, в которых есть много пропусков

In [ ]:
for column in meta.columns:
    print(f'Column name: {column} {round(meta[column].notna().sum() / len(meta) * 100, 2)}%')
    print(meta[column].value_counts() if len(meta[column].unique()) < 14 else f'so much unique values\n{meta[column].describe()}')
    print()

In [ ]:
"""
TODO
Хотелось бы поподробнее о таких столбцах, как
static_noise, burst_noise, extra_beats, start_fold, validated_by
"""

На основе этих данных выделим список столбцов с множество недостающих значений

In [ ]:
empty_columns = ['height', 'weight', 'heart_axis']
meta.drop(columns=empty_columns, inplace=True)

Удалим те строки, в которых electrodes_problem

In [ ]:
meta.drop(meta[meta['electrodes_problems'].notna()].index, inplace=True)
meta.drop(columns=['electrodes_problems'], inplace=True)
meta.drop(1514, inplace=True) # Хех пока

In [ ]:
meta = meta.reset_index(drop=True)

In [ ]:
meta.info()

In [ ]:
"""
TODO удалить бесполезные записи
Сначала их надо найте, это такие записи, где либо много пропущенных столбцов
Либо те, что с проблемным экг
"""

Отдельно рассмотрим столбец report, т.к. его нужно по особому обрабатывать

In [ ]:
reports = [report.strip('. ').split('. ') for report in meta['report'].unique()]

In [ ]:
"""
TODO Непонятно что делать с репортами
Возможно их стоит просто отбросить,
Возможно их стоит по умному обрабатывать,
Но у них еще есть неточности по типу поставил/не поставил точку в конце - это мешает их корректно обрабатывать
"""
subreports = set()

for report in reports:
    subreports = subreports.union(set([i.strip(',. ') for i in report]))

print(len(subreports))
pprint(subreports)

Заменим строки в названиях, так как они малину портят

In [ ]:
str_columns = ['infarction_stadium1', 'infarction_stadium2', 'burst_noise', 'pacemaker']

meta = pd.get_dummies(meta, columns=str_columns)

In [ ]:
"""
TODO у нас еще остались столбцы baseline_drift static_noise extra_beats, scp_codecs
"""
meta.info()

...

## Тестирование различных моделей

Используемые модули (не точно)

In [ ]:
# Нейронки (какие-то уберем, разобраться сначала, какие для этого подходят)
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.svm import SVR

# Посложнее, что успеем сделать
import torch #1
import keras #2

# Метрики
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score